# Scores voor vakken voorspellen

We geven een lijst van studenten aan een docent en vragen hoe hij/zij deze studenten inschat.
De docent mag aan iedere student een score toekennen volgens zijn gevoel (niet volgens punten).
We verwachten dat een docent dit onbewust toch zal doen op basis van zijn eigen vakken.


1. Lees het bestand "studentenScores.csv" in.  Hierin zie je de subjectieve scores, gegeven door de docent.  De score kan zijn: "goed", "gemiddeld", "slecht".  We vragen ons nu af op basis van welke criteria de docent zijn scores heeft gegeven.

In [1]:
import numpy as np
import pandas as pd
import graphviz
from id3 import Id3Estimator, export_graphviz

studentenScores = pd.read_csv('data/studentenScores.csv')
display(studentenScores)

,vak1,vak2,vak3,vak4,vak5,vak6,vak7,vak8,vak9,score
0,13,9,17,2,17,1,0,14,18,gemiddeld
1,10,13,5,20,20,14,4,2,10,goed
2,13,4,2,19,3,9,16,13,7,goed
3,8,3,17,7,16,10,15,9,8,slecht
4,1,12,13,2,9,10,2,2,13,slecht
...,...,...,...,...,...,...,...,...,...,...
95,14,10,15,4,4,6,10,13,8,gemiddeld
96,16,10,16,1,8,11,19,0,13,gemiddeld
97,0,20,9,7,13,11,18,10,9,slecht
98,14,15,16,9,4,8,10,18,9,gemiddeld


2. Stel daartoe een beslissingsboom op voor de score.

3. Welke vakken geeft de docent?

4. De boomstructuur is nogal complex.

Dat komt omdat de punten veel waarden kunnen aannemen.
We kunnen dit oplossen door de punten op te delen in categorieën:
 * niet geslaagd (0-9),
 * voldoening (10-13),
 * onderscheiding (14-15),
 * grote onderscheiding (16-20).

Probeer de scores zo in te delen.


5. Bereken nu de boomstructuur opnieuw.  Is dit duidelijker?

6. Hoe zou de docent iemand inschatten als die volgende punten heeft: 9, 14, 13, 12, 15, 13, 7, 10, 18

7. Maak eens een boomstructuur met het C4.5 algoritme en een ScitKit DecisionTree op de originele gegevens en kijk welk resultaat daar uit komt.

8. Maak eens een boomstructuur met een ScitKit DecisionTree op de onehot encoded gegevens (gebruik Pandas `get_dummies`) en kijk welk resultaat daar uit komt.